# Loading packages

In [1]:
#load pacakges 
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install transformers
!pip install tqdm
!pip install torch 
!pip install gensim



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


# Loading, cleaning and filtering data

In [2]:
import pandas as pd 
import re

# Define the dtype dictionary
dtype_dict_all = {
    'conversation_id': 'object',
    'lang': 'object',
    'created_at': 'object',
    'id': 'object',
    'text': 'object',
    'author_id': 'object',
    'referenced_tweets_id': 'object',
    'referenced_tweets_type': 'object',
    'in_reply_to_user_id': 'object',
    'PNR': 'object',
    'surveyXact_externke': 'object',
    'cleaned_text': 'object'
}

# Load in the scored csv file with the dictionary
scored_tweets= pd.read_csv('/Users/idahelenedencker/Desktop/STANDBY_Ida/Rec-nition and attack on twitter data/CSV files/twitter_w_rec_attack_final.csv',  dtype=dtype_dict_all)

#check data types
scored_tweets.dtypes 


conversation_id                  object
lang                             object
created_at                       object
id                               object
text                             object
author_id                        object
public_metrics.retweet_count      int64
public_metrics.reply_count        int64
public_metrics.like_count         int64
public_metrics.quote_count        int64
referenced_tweets_id             object
referenced_tweets_type           object
in_reply_to_user_id              object
__index_level_0__               float64
PNR                              object
surveyXact_externke              object
non_unique_twitter_author_id    float64
started_survey                  float64
cleaned_text                     object
rec-nition                        int64
attack                            int64
dtype: object

In [3]:
#convert non_unique_twitter_author_id and started_survey into integer
scored_tweets['non_unique_twitter_author_id'] = pd.to_numeric(scored_tweets['non_unique_twitter_author_id'], errors='coerce').astype('Int64')
scored_tweets['started_survey'] = pd.to_numeric(scored_tweets['started_survey'], errors='coerce').astype('Int64')

#check value counts
scored_tweets['rec-nition'].value_counts(dropna=False) 
scored_tweets['attack'].value_counts(dropna=False) 

#filter to replied_to
replies = scored_tweets[scored_tweets['referenced_tweets_type'] == 'replied_to']


In [ ]:
replies

- conversation_id: the Post ID of the original Post that started the conversation. (if the post is not a Retweet, Quoted Tweet, or reply the conversation_id will be the same as id)
- lang: language
- created_at: when it was created (YYYY-MM-DDTHH:MM:SS.000Z, ISO8601 format giving: year, month, data at Time: hour:minute:second:milisecond in Coordinated Universal Time (UTC))
- id: Each object within Twitter - a Tweet, Direct Message, User, List, and so on - has a unique ID
- text: the post, starts with @ if it is a retweet or reply to
- author_id: returns user object representing the Post's author
- public_metrics.retweet_count: how many retweeted
- public_metrics.reply_count: how many replied
- public_metrics.like_count: how many liked
- public_metrics.quote_count: how many quoted
- referenced_tweets_id: returns post object that this Post is referencing (either as a Retweet, Quoted Tweet, or reply)
- referenced_tweets_type: if it is a post (None), retweeted or replied_to etc.
- in_reply_to_user_id: returns a user object representing the Post author this requested Post is a reply of
- __index_level_0__: ??
- PNR: personnummer (Eentydig identifikation af person), like cpr
- surveyXact_externke: personification ID linked to the e-boks survey
- non_unique_twitter_author_id (made by simon): ?? can take 0 or 1
- started_survey: (made by simon): if they started the e-boks survey (doesnt mean finished it) can take 0 or 1



Read more here: 
- https://developer.x.com/en/docs/twitter-ids (ID)
- https://developer.x.com/en/docs/twitter-api/expansions (in_reply_to_user_id, author_id, referenced_tweets_id)
- https://developer.x.com/en/docs/twitter-api/conversation-id (conversation_id)

In [ ]:
# look at performance of algorithms

#pull up text examples
attack= replies[replies['attack'] == 1]
print(attack['text'][:100])

rec_nition= replies[replies['rec-nition'] == 1]
print(rec_nition['text'][:100])

# can print the whole text using
text_to_print = rec_nition['text'].head(20).tolist()
text_to_print

# Limiting the df

## Random sampling (10k samples)

In [ ]:
#random sampling
replies_random= replies.sample(10000)
replies_random

## Stratified sampling 

### Using attack/regnition split

In [ ]:
# stratified sampling using attack and rec-nition 

#idea: ¼ are scored 0-0 for attack-recnition; ¼ are scored 1-0 for attack-recnition, ¼ are scored 0-1 for attack-recnition, ¼ are scored 1-1 (possible that there are no/few examples that have both attack AND rec-nition labels) for attack-recnition 

# First: Check if there are any 1-1 cases
condition = (replies['attack'] == 1) & (replies['rec-nition'] == 1)
rows_with_condition = replies[condition]
rows_with_condition #1156 cases (but not enough to be 25%)

#new idea: 1000 of 1-1, 3000 1-0, 3000 0-1, 3000 0-0

#make a new column 'A/R' with both scorings can take 0-0, 0-1, 1-1, or 1-0
replies['A/R'] = replies['attack'].astype(str) + '-' + replies['rec-nition'].astype(str)

#check how many of each
replies['A/R'].value_counts(dropna=False) 

#Make a new dataframe with the new split 
con_1 = (replies['A/R'] == '1-1')
df1_1 = replies[con_1].sample(1000)

con_2 = (replies['A/R'] == '1-0')
df1_0 = replies[con_2].sample(3000)

con_3 = (replies['A/R'] == '0-0')
df0_0 = replies[con_3].sample(3000)

con_4 = (replies['A/R'] == '0-1')
df0_1 = replies[con_4].sample(3000)

# Concatenate the DataFrames vertically
stratified_sample = pd.concat([df1_1, df1_0,df0_0,df0_1], axis=0)
stratified_sample


### Using word embeddings

In [8]:
import string

#Remove user data
#e.g '@ninna1122 i dont agree with you' will be 'i dont agree with you'
replies['preprocessed_text'] = replies['cleaned_text'].apply(lambda x: re.sub(r'@\w+', '', x).strip())

#lowercase
replies['preprocessed_text'] = replies['preprocessed_text'].str.lower()

#remove special charachters 
replies['preprocessed_text'] = replies['preprocessed_text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))



/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_6347/2117740019.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  replies['preprocessed_text'] = replies['cleaned_text'].apply(lambda x: re.sub(r'@\w+', '', x).strip())
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_6347/2117740019.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  replies['preprocessed_text'] = replies['preprocessed_text'].str.lower()
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_6347/2117740019.py:11: S

In [ ]:
replies

In [10]:
# test df
test = replies.head(20000)

#### TF-idf

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#cosine similarity to hate sentence ('Du for dum at høre på. Sikke en idiot. Jeg hader dig')

v= TfidfVectorizer()
X = v.fit_transform(replies['preprocessed_text'])

# Transform the snetence
sentence = "du for dum at høre på sikke en idiot jeg hader dig"
sentence_tfidf = v.transform([sentence])

# Calculate cosine similarity between the sentence and each sentence
cosine_similarities = cosine_similarity(X, sentence_tfidf)

# Output the cosine similarities
replies['tfidf_cos_sim_hate_sentence'] = cosine_similarities

# Sort the DataFrame by cosine similarity in descending order
sorted_replies = replies.sort_values(by='tfidf_cos_sim_hate_sentence', ascending=False)

# Select the top 4000 entries
top_replies_hate = sorted_replies.head(4000)

# Print the results and score
#print(top_replies_hate[['text', 'tfidf_cos_sim_hate_sentence']].head(10))

#print the whole text
text_to_print = top_replies_hate['preprocessed_text'].head(20).tolist()
text_to_print


/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_6347/3244366749.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  replies['tfidf_cos_sim_hate_sentence'] = cosine_similarities


['du dum at høre på',
 'du er dum at høre på',
 'du er så dum at høre på',
 'du er jo bare dum at høre på',
 'hvor er du dog dum at høre på',
 'jeg hader dig',
 'jeg hader dig',
 'jeg hader dig',
 'at du er dum at høre på er ikke  nødvendigvis  det samme som at du er dum',
 'nej det er jeg ikke men du er stadig dum at høre på',
 'jeg hader dig ikke💐😊😘',
 'jeg hader dig ikke ',
 'sikke en egoistisk idiot 🙄',
 'nu er du bare dum at høre på og du ved det nok godt',
 '😶 du er dum',
 'du er dum',
 'fuck jeg hader dig \njeg æder det jo ikke din idiot',
 'er du dum eller hvad er jeg også træt af at høre på',
 'du er virkelig dum at høre på ole',
 'sikke en 😊']

In [12]:
#cosine similarity to prosocial sentence ('Jeg forstår din frustration, men. Lad os huske den gode tone..Jeg finder ikke din kommentar særlig konstruktiv. Vi har måske forskellige synspunkter, men.. Had bidrager ikke til den gode debat. Dette er ikke sandt, det er faktuelt forkert, du misforstår hvad.. ')

v= TfidfVectorizer()
X = v.fit_transform(replies['preprocessed_text'])

# Transform the sentence
sentence = "jeg forstår din frustration men lad os huske den gode tone jeg finder ikke din kommentar særlig konstruktiv vi har måske forskellige synspunkter men had bidrager ikke til den konstruktive debat dette er ikke sandt det er faktuelt forkert du misforstår hvad"
sentence_tfidf = v.transform([sentence])

# Calculate cosine similarity between the sentence and each sentence
cosine_similarities = cosine_similarity(X, sentence_tfidf)

# Output the cosine similarities
replies['tfidf_cos_sim_prosocial_sentence'] = cosine_similarities

# Sort the DataFrame by cosine similarity in descending order
sorted_replies = replies.sort_values(by='tfidf_cos_sim_prosocial_sentence', ascending=False)

# Select the top 4000 entries
top_replies_prosocial = sorted_replies.head(4000)

# Print the results and score
#print(top_replies_prosocial[['text', 'tfidf_cos_sim_prosocial_sentence']].head(10))

#print the whole text
text_to_print = top_replies_prosocial['preprocessed_text'].head(20).tolist()
text_to_print


/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_6347/1586220956.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  replies['tfidf_cos_sim_prosocial_sentence'] = cosine_similarities


['så forstår jeg ikke din kommentar',
 'det er faktuelt forkert',
 'det skriver jeg nu heller ikke ikke forkert men skal vi sige bidrager til ',
 'jeg forstår godt din frustration og din pointe det er slet ikke det httpstco8wvbvpuuv9',
 'forstår ikke  eller misforstår',
 'hvad bidrager du mon til med den kommentar tænker du',
 'den gode tone du ved 😉',
 'den kommentar forstår jeg ikke helt hvad er der galt',
 'nu er der mig der ikke forstår  altså hvad betyder din kommentar',
 'det er faktuelt forkert det du skriver',
 'er det din🤔',
 'er det din 😊',
 'er det din',
 'er det din',
 'er det din🙄',
 'er det din 😘',
 'er det din',
 'er det din 😉',
 'er det din ❤️',
 'er det din 😳']

#### Pre-tranied BERT-model (on prosocial sentence)

In [13]:
#Performs quite allright on the test data: but slow (7 hr on whole dataset)

from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Function to get BERT embeddings
def get_bert_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()


# Apply tqdm to the apply function to track progress
tqdm.pandas()

# Use the apply function with progress tracking
sentence_embeddings = test['preprocessed_text'].progress_apply(lambda x: get_bert_embedding(x, tokenizer, model))

# Get embedding for hate sentence
sentence ="jeg forstår din frustration men lad os huske den gode tone jeg finder ikke din kommentar særlig konstruktiv vi har måske forskellige synspunkter men had bidrager ikke til den konstruktive debat dette er ikke sandt det er faktuelt forkert du misforstår hvad"
prosocial_sentence_embedding = get_bert_embedding(sentence, tokenizer, model).reshape(1, -1)

# Calculate cosine similarity between the word "politik" and each sentence with progress tracking
cosine_similarities = [cosine_similarity([embedding], prosocial_sentence_embedding)[0][0] for embedding in tqdm(sentence_embeddings, desc="Calculating Cosine Similarities")]

# Output the cosine similarities
test['BERT_cos_sim_prosocial_sentence'] = cosine_similarities

# Sort the DataFrame by cosine similarity in descending order
sorted_replies = test.sort_values(by='BERT_cos_sim_prosocial_sentence', ascending=False)

# Select the top 15 entries
top_replies = sorted_replies.head(30)

# Print the results
#print(top_replies[['text', 'BERT_cos_sim_hate_sentence']])

#print the whole text
text_to_print = top_replies['preprocessed_text'].head(20).tolist()
text_to_print



Calculating Cosine Similarities: 100%|██| 20000/20000 [00:02<00:00, 9026.30it/s]
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_6347/4066471403.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['BERT_cos_sim_prosocial_sentence'] = cosine_similarities


['altså jeg er enig i det du kritiserer men det er vel strengt taget ikke det han tweeter som jeg læser det bruger han anledningen ck til at romantisere ikke en sandsynlighedsberegning',
 'nej jeg tror ikke du misforstår og jeg forstår også godt hvad du mener på et teoretisk plan fungerer dit svar også 100 jeg problematiserer det også mere på et praktisk niveau det var bare en lille brain fart der overkom mig',
 'er det et langsigtet behov at gøre størstedelen af befolkningen afhængig af det offentlige jeg forstår slet ikke din argumentation for mig at se er velfærd lig med skumfidusen og vi spiser den alle sammen  med det samme',
 'sikke da et plat og i øvrigt forkert tweet vi er ikke tavse vi er forargede vi er imod snyd og kriminalitet lad venligst være med at påstå andet',
 'lige en lille opfordring vil du ikke nok sætte dig bare lidt ind i tingene inden du fortsætter med komme med udokumenterede påstande \ndet er helt ok at have en anden mening men at du konsekvent spreder den ene

#### XLM-RoBERTa (on prosical sentence)

In [14]:
#Decent on the test data (takes also 7hr on full dataset)

from transformers import RobertaTokenizer, RobertaModel

# Load RoBERTa model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')


# Function to get RoBERTa embeddings
def get_roberta_embedding(sentence, tokenizer, model):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()


# Generate embeddings for DataFrame
embeddings = [get_roberta_embedding(text, tokenizer, model) for text in tqdm(test['preprocessed_text'], desc="Generating Embeddings")]


# Get embedding for hate sentence
sentence = "jeg forstår din frustration men lad os huske den gode tone jeg finder ikke din kommentar særlig konstruktiv vi har måske forskellige synspunkten men had bidrager ikke til den konstruktive debat dette er ikke sandt det er faktuelt forkert du misforstår hvad"
prosocial_sentence_embedding = get_roberta_embedding(sentence, tokenizer, model).reshape(1, -1)

# Calculate cosine similarity
test['roberta_cos_sim_prosocial_sentence'] = [cosine_similarity([embedding], prosocial_sentence_embedding)[0][0] for embedding in tqdm(embeddings, desc="Calculating Cosine Similarities")]


# Sort and display top results
sorted_df = test.sort_values(by='roberta_cos_sim_prosocial_sentence', ascending=False)
top_replies = sorted_df.head(30)

#print(top_replies[['text', 'roberta_cos_sim_hate_sentence']])

#print the whole text
text_to_print = top_replies['preprocessed_text'].head(20).tolist()
text_to_print



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Calculating Cosine Similarities: 100%|██| 20000/20000 [00:02<00:00, 8095.84it/s]
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_6347/1634502232.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['roberta_cos_sim_prosocial_sentence'] = [cosine_similarity([embedding], prosocial_sentence_embedding)[0][0] for embedding in tqdm(embeddings, desc="Calculating Cosine Similarities")]


['almindelige mennesker har heller ikke råd til at bo alle steder jeg gad godt bo på strandvejen men det har jeg ikke råd til skal vi så til at lave en lov om det \n\ndesuden kan du ikke renovere lejlighed hvor du bor i forvejen  det kun når den står tom muligheden opstår',
 'jeg stiller altid op hvornårsomhelst til debat ang iran men der er ik ngt at misforstå her jeg synes det er et meget relevant og vigtigt spm at stille om der var specifik trussel mod danske skibe før snak om maritim indsats det gør puck ikke',
 'det er dig der retweeter mig bygger en stråmand og så nægter at forholde dig til substansen når jeg tager mig tid til at svare men det var da godt at nogen tager det alvorligt  så basalt et spørgsmål af så stor betydning utroligt du som analytiker ikke kan forstå det',
 'jeg tror godt du forstår minimumsnormeringer er politisk længere er den ikke når du smider kortet om at nogen må give sig så er det fordi forslaget er politisk \nmin pointe er institutioner ikke bør styres

#### Fasttext (an extension of Word2Vec) 

In [15]:
#Hate sentence embeddings, on whole dataset

from gensim.models.fasttext import load_facebook_model
import numpy as np

# Load the pretrained FastText model: takes 5 min to load: uncomment after first run
fasttext_model = load_facebook_model('/Users/idahelenedencker/Desktop/STANDBY_Ida/Creating dataset of reference tweets/Models/cc.da.300.bin')

# Function to get FastText embeddings
def get_fasttext_embedding(sentence, model):
    words = sentence.split()
    word_embeddings = [model.wv[word] for word in words if word in model.wv]
    if not word_embeddings:  # If no words in the sentence are in the vocabulary
        return np.zeros(model.vector_size)
    sentence_embedding = np.mean(word_embeddings, axis=0)
    return sentence_embedding

# Generate embeddings for DataFrame
embeddings = [get_fasttext_embedding(text, fasttext_model) for text in tqdm(replies['preprocessed_text'], desc="Generating Embeddings")]

# Get embedding for hate sentence
hate_sentence = "du for dum at høre på sikke en idiot jeg hader dig"
hate_sentence_embedding = get_fasttext_embedding(hate_sentence, fasttext_model).reshape(1, -1)

# Calculate cosine similarity
replies['fasttext_cos_sim_hate_sentence'] = [cosine_similarity([embedding], hate_sentence_embedding)[0][0] for embedding in tqdm(embeddings, desc="Calculating Cosine Similarities")]

# Sort and display top results
sorted_df = replies.sort_values(by='fasttext_cos_sim_hate_sentence', ascending=False)
top_replies_hate = sorted_df.head(4000)

#print(top_15_replies[['text', 'fasttext_cos_sim_hate_sentence']])

#print the whole text
text_to_print = top_replies_hate['preprocessed_text'].head(20).tolist()
text_to_print

Generating Embeddings: 100%|█████████| 642613/642613 [00:56<00:00, 11461.60it/s]
Calculating Cosine Similarities: 100%|█| 642613/642613 [01:17<00:00, 8241.49it/s
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_6347/2306015175.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  replies['fasttext_cos_sim_hate_sentence'] = [cosine_similarity([embedding], hate_sentence_embedding)[0][0] for embedding in tqdm(embeddings, desc="Calculating Cosine Similarities")]


['du er sgu endnu engang for dum til at jeg gider at bruge tid på dig',
 'du har da ikke selv holdt dig for god til at anmelde hvad du vurderede at være en falsk reklame så måske skulle du finde en mindre hest at sidde på',
 'nu slukker jeg sgu for dig for du er virkelig irriterende at høre på',
 'nu er du bare dum at høre på her troede man at man kunne have en alm diskussion med en voksen mand trist',
 'shit da en røvlort \njeg er fandeme ked af at du har set dig nødsaget til at gøre dette og gal for helvede men jeg håber det betyder at du får en bedre oplevelse her\nsikke en lortestodder',
 'at du sender mig en tanke når du ser en smuk pige spille fiol har jeg intet ondt at sige om \ngodmorgen',
 'håber du er okay lad mig vide hvis du skal bruge en skulder at græde på',
 'du er fanme da for dum at høre på jeg håber da at ulven tager sig kærligt af dig og dine så får piben en anden lyd',
 'jeg tror du tvivler på alt fordi du synes den her kamp er en du lige skal blande dig i uden at t

In [16]:
#Prosocial sentence embeddings, on whole dataset


# Function to get FastText embeddings
def get_fasttext_embedding(sentence, model):
    words = sentence.split()
    word_embeddings = [model.wv[word] for word in words if word in model.wv]
    if not word_embeddings:  # If no words in the sentence are in the vocabulary
        return np.zeros(model.vector_size)
    sentence_embedding = np.mean(word_embeddings, axis=0)
    return sentence_embedding

# Generate embeddings for DataFrame
embeddings = [get_fasttext_embedding(text, fasttext_model) for text in tqdm(replies['preprocessed_text'], desc="Generating Embeddings")]

# Get embedding for hate sentence
prosocial_sentence = "jeg forstår din frustration men lad os huske den gode tone jeg finder ikke din kommentar særlig konstruktiv vi har måske forskellige synspunkten men had bidrager ikke til den konstruktive debat dette er ikke sandt det er faktuelt forkert du misforstår hvad"
prosocial_sentence_embedding = get_fasttext_embedding(prosocial_sentence, fasttext_model).reshape(1, -1)

# Calculate cosine similarity
replies['fasttext_cos_sim_prosocial_sentence'] = [cosine_similarity([embedding], prosocial_sentence_embedding)[0][0] for embedding in tqdm(embeddings, desc="Calculating Cosine Similarities")]

# Sort and display top results
sorted_df = replies.sort_values(by='fasttext_cos_sim_prosocial_sentence', ascending=False)
top_replies_prosocial = sorted_df.head(4000)

#print(top_15_replies[['text', 'fasttext_cos_sim_hate_sentence']])

#print the whole text
text_to_print = top_replies_prosocial['preprocessed_text'].head(20).tolist()
text_to_print

Generating Embeddings: 100%|█████████| 642613/642613 [00:51<00:00, 12401.56it/s]
Calculating Cosine Similarities: 100%|█| 642613/642613 [01:16<00:00, 8429.34it/s
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_6347/449282597.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  replies['fasttext_cos_sim_prosocial_sentence'] = [cosine_similarity([embedding], prosocial_sentence_embedding)[0][0] for embedding in tqdm(embeddings, desc="Calculating Cosine Similarities")]


['jeg siger det vel strengt taget bare til dem der følger mig  men hvis du følger den efterfølgende diskussion så er jeg netop ikke imponeret over den tilgang spillerne har',
 'had alt det du vil men du skal ikke gøre dig til dommer over hvad jeg mener er en naturlig reaktion jeg dækker overhovedet ikke den sag',
 'din første indvending er muligvis korrekt det skal vurderes den anden forstår jeg ikke hvilke andre vacciner tilbyder vi offlabel',
 'hvis det ovenikøbet er er kendt adfærd så fortsætter den jo udelukkende fordi ingen tager den videre  han kan jo desværre ikke se det  hvis du ikke ønsker den store sag  så tror jeg du skal gøre mere ved selve konfrontationen med ham  giv din vrede meregas  for din skyld ♥️',
 'det skrev jeg ikke men uanset hvad jeg kaster efter dig vil du jo ikke godtage det  jeg har ikke en blind tro men jeg vurdere efter bedste evne at man har gjort det rigtige med fejl og alt muligt undervejs klart men grundlæggende den rigtige respons',
 'tror det er den 

### Using Fasttext embeddings (20 k sample)

In [17]:
# Make a 8000/8000/4000 split: using fasttext

#delete tfidf_cos_sim_hate_sentence and tfidf_cos_sim_prosocial_sentence columns
replies = replies.drop(['tfidf_cos_sim_hate_sentence', 'tfidf_cos_sim_prosocial_sentence'], axis=1)

#make dataframes based on cosine similarity scores 
sorted_df_prosocial = replies.sort_values(by='fasttext_cos_sim_prosocial_sentence', ascending=False).head(8000)
sorted_df_hate = replies.sort_values(by='fasttext_cos_sim_hate_sentence', ascending=False).head(8000)

#make random dataframe
sorted_df_random = replies.sample(4000)

# Concatenate the DataFrames vertically
word_embeddings_fasttext = pd.concat([sorted_df_prosocial, sorted_df_hate,sorted_df_random], axis=0)
word_embeddings_fasttext

#save
word_embeddings_fasttext.to_csv('/Users/idahelenedencker/Desktop/STANDBY_Ida/Creating dataset of reference tweets/replies_sample.csv', index=False)